# VGG Finetuning
- https://www.programcreek.com/python/example/108009/torchvision.models.vgg16

In [9]:
!pip install torchsummary

In [15]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torchsummary import summary

In [11]:
class VGG16Net(nn.Module):
    def __init__(self, embedding_size):
        super(VGG16Net, self).__init__()
        model = models.vgg16(pretrained=False)  #False

        self.features = model.features
        for param in self.features.parameters(): #NOTE: prune:True  // finetune:False
            param.requires_grad = True    #grad를 계산하겠다는 의미, 기본적으로 True  (False: 원래 것 사용하겠다는 의미)

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(25088, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
        )
        self.cls_fc = nn.Linear(4096, 31) 
        

    def forward_once(self, x):
        out = self.features(x)
        out = self.classifier(out)
        out = self.cls_fc(out)

        return outp

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [12]:
model = VGG16Net(512)

In [13]:
summary(model, [(3, 224, 224),(3, (224, 224))])

TypeError: rand(): argument 'size' must be tuple of ints, but found element of type tuple at pos 3

### 실제 소스에서는 실행됨

# Squeezenet

In [18]:
dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'quantization',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'utils',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vg

In [19]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        model = models.squeezenet(pretrained=False)  #False

        self.features = model.features
        for param in self.features.parameters(): #NOTE: prune:True  // finetune:False
            param.requires_grad = True

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Conv2d(512, 1000, kernel_size=(1, 1), stride=(1, 1)),
            nn.ReLU(inplace),
            nn.AvgPool2d(kernel_size=13, stride=1, padding=0),
            )
        self.cls_fc = nn.Linear(1000, 512) 


    def forward_once(self, x):
        out = self.features(x)
        out = self.classifier(out)
        out = self.cls_fc(out)


    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2    

In [20]:
model = SiameseNetwork().cuda()

TypeError: 'module' object is not callable

In [21]:
summary(model, [(3, 224, 224),(3, (224, 224))])

TypeError: rand(): argument 'size' must be tuple of ints, but found element of type tuple at pos 3